In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 1.date read & cleansing

In [2]:
train_org=pd.read_csv("/kaggle/input/machine-learning-on-titanic-data-set/train.csv") 
test_org=pd.read_csv("/kaggle/input/machine-learning-on-titanic-data-set/test.csv")
gender_submission=pd.read_csv("/kaggle/input/machine-learning-on-titanic-data-set/gender_submission.csv")

In [3]:
#train_org.sum() # 총 889 행, 12컬럼, Age, cabin 컬럼에 널값이 있음., Age 평균 29
train_org.info()


In [4]:
# test_org.sum() # 총 418행 11컬럼, Survived 컬럼 외.
test_org.info()

In [5]:

# Age 널값 보정
train_org['Age'].fillna(value=train_org['Age'].mean(),inplace=True)
test_org['Age'].fillna(value=test_org['Age'].mean(),inplace=True)


# Embarked 상수화
train_org.loc[train_org['Embarked']=='Q','Embarked']=1
train_org.loc[train_org['Embarked']=='S','Embarked']=2
train_org.loc[train_org['Embarked']=='C','Embarked']=0


test_org['Embarked']=test_org['Embarked'].map({'Q':1,'S':2,'C':0})

# 성별 값을 상수화 
train_org['Sex'] = train_org['Sex'].replace('male',0)
train_org['Sex'] = train_org['Sex'].replace('female',1)

test_org['Sex'] = test_org['Sex'].map({'male':0,'female':1})

# train['Age'].plot(kind="hist")

# Fare 널값 보정
test_org['Fare'].fillna(value=test_org['Fare'].mean(),inplace=True)


# Embarked 널값 제거
train_org.dropna(subset=['Embarked'],inplace=True)
test_org.dropna(subset=['Embarked'],inplace=True)

In [6]:
test_org.describe() # 기초 통계값 출력

### 2.피어슨 상관계수 출력 

In [7]:
train = train_org.copy()

train_org.query('Age == "Nan"')
train_org[train_org['Age'].isnull()]
test_org.groupby('Embarked').count()

In [8]:
train['Embarked'] = train['Embarked'].astype(int)

In [9]:
#train=train.drop('Embakred',axis=1)
train.corrwith(train['Fare'])

In [10]:
train.corr()

### 3.EDA

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.displot(x='Age',data=train_org, col='Survived',hue='Survived')
plt.show()


In [12]:
sns.displot(x='Sex',data=train_org,col='Survived',hue='Survived')
plt.show()

### 4. sklearn 모델 기반 예측 및 출력

In [13]:
x_train = train_org[['Sex','Fare']]
y_train = train_org['Survived']
#test_org.dropna(subset=['Fare'],inplace=True)
x_test = test_org[['Sex','Fare']]

#train_org=train_org.drop(['Survived'],axis=1)

In [14]:
test_org.info()
x_test = test_org[['Sex','Fare']]
#x_train,x_test,y_train,y_test=train_test_split(train_org,test_org,test_size=0.2,random_state=42)

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

# x_train=x_train.reset_index()
# x_test = x_test.reset_index()

In [16]:
x_train.shape,y_train.shape,x_test.shape

In [17]:
model.fit(x_train,y_train)

In [18]:
#x_test.info()
y_pred = model.predict(x_test)

In [21]:
submission = pd.DataFrame({    # 제출 데이터 포맷으로 df 생성
        "PassengerId": test_org["PassengerId"],
        "Survived": y_pred
    })

In [20]:
submission.to_csv("titanic.csv",index=False)   # 제출 데이터 출력